In [1]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

from chromadb import Documents, EmbeddingFunction, Embeddings

from google import genai

client = genai.Client(api_key='')


for m in client.models.list():
  if 'embedContent' in m.supported_actions:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [2]:
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    EMBEDDING_MODEL_ID = "models/embedding-001"
    title = "Custom query"
    response = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=input,
        config=types.EmbedContentConfig(
          task_type="retrieval_document",
          title=title
        )
    )
    # 모든 문서의 임베딩 벡터를 리스트로 반환
    return [e.values for e in response.embeddings]

In [3]:
import json
#chroma_client.delete_collection("my_collection")
def preprocess_metadata(metadata):
    new_metadata = {}
    for k, v in metadata.items():
        if isinstance(v, list):
            new_metadata[k] = ", ".join(map(str, v))  # 리스트를 문자열로 변환
        else:
            new_metadata[k] = v
    return new_metadata
def batch_add(collection, documents, metadatas, ids, batch_size=100):
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i+batch_size]
        batch_metas = metadatas[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]
        collection.add(
            documents=batch_docs,
            metadatas=batch_metas,
            ids=batch_ids
        )
def create_chroma_db(json_data, name):
    import chromadb

    chroma_client = chromadb.Client()
    db = chroma_client.create_collection(
        name=name,
        embedding_function=GeminiEmbeddingFunction()
    )

    # JSON 데이터에서 텍스트, 메타데이터, id 추출
    documents = [item["text"] for item in json_data]
    metadatas = [preprocess_metadata(item["metadata"]) for item in json_data]
    ids = [str(i) for i in range(len(json_data))]
    
    batch_add(db, documents, metadatas, ids, batch_size=100)
   
    return db
with open('disease_rag_with_metadata.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
db = create_chroma_db(data, "my_collection")

C:\Users\eh589\AppData\Local\Temp\ipykernel_9752\2476913475.py:27: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GeminiEmbeddingFunction()


In [4]:
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:3],
    "Documents": sample_data['documents'][:3],
    "Embeddings": [str(emb)[:50] + "..." for emb in sample_data['embeddings'][:3]]  # Truncate embeddings
})

print(df)

  IDs                                          Documents  \
0   0  Fever, Fatigue, Difficulty Breathing 증상이 있는 경우...   
1   1      Cough, Fatigue 증상이 있는 경우 Common Cold일 수 있습니다.   
2   2           Cough, Fatigue 증상이 있는 경우 Eczema일 수 있습니다.   

                                          Embeddings  
0  [ 3.16955782e-02 -4.14985903e-02 -4.31520194e-...  
1  [ 7.59642050e-02 -6.42082170e-02 -7.00410604e-...  
2  [ 6.10879771e-02 -5.23152687e-02 -7.92896152e-...  


In [13]:
def get_relevant_passage(query, db, n_results=5):
  passages = db.query(query_texts=[query], n_results=n_results)['documents'][0]
  return passages
# Perform embedding search
passages = get_relevant_passage("Fever, Cough, Difficulty Breathing", db, 5)
print(passages)

['Fever, Fatigue, Difficulty Breathing 증상이 있는 경우 Asthma일 수 있습니다.', 'Fever, Fatigue, Difficulty Breathing 증상이 있는 경우 Asthma일 수 있습니다.', 'Fever, Fatigue 증상이 있는 경우 Asthma일 수 있습니다.', 'Fever, Cough, Fatigue, Difficulty Breathing 증상이 있는 경우 Asthma일 수 있습니다.', 'Fever, Cough, Fatigue, Difficulty Breathing 증상이 있는 경우 Asthma일 수 있습니다.']


In [ ]:
def make_prompt(query, relevant_passages):
  escaped = " ".join([p.replace("'", "").replace('"', "").replace("\n", " ") for p in relevant_passages])
  prompt = f"""
  당신은 사용자의 증상에 대해 의학적으로 도움을 주는 상담 도우미입니다.

  아래는 참고할 수 있는 임상 데이터입니다:
  - 사용자 질문 (QUESTION): "{query}"
  - 관련 정보 (PASSAGE): "{escaped}"

이 정보를 참고하여 사용자에게 친절하고 정확하게 답변해 주세요.
또한 병원 방문 및 전문적인 상담을 권유하지 마세요.
또한 PASSAGE에 영어 단어가 포함되어 있다면, 괄호 안에 한글 뜻을 함께 제공해 주세요.

답변은 아래 ANSWER: 이후에 이어서 작성해 주세요.

ANSWER:
""".format(query=query, relevant_passages=escaped)
  return prompt
query = "나는 기침을 자주하고 요즘 콧물도 나는 것 같아"
passages = get_relevant_passage(query, db, 5)
prompt = make_prompt(query, passages)
print(prompt)
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-05-06"] {"allow-input": true, "isTemplate": true}
answer = client.models.generate_content(
    model = MODEL_ID,
    contents = prompt
)
print(answer.text.split("ANSWER:"[0]))


  당신은 사용자의 증상에 대해 의학적으로 도움을 주는 상담 도우미입니다.

  아래는 참고할 수 있는 임상 데이터입니다:
  - 사용자 질문 (QUESTION): "나는 기침을 자주하고 요즘 콧물도 나는 것 같아"
  - 관련 정보 (PASSAGE): "Fatigue 증상이 있는 경우 Depression일 수 있습니다. Fatigue 증상이 있는 경우 Depression일 수 있습니다. Cough 증상이 있는 경우 Eczema일 수 있습니다. Cough 증상이 있는 경우 Eczema일 수 있습니다. Fatigue 증상이 있는 경우 Dementia일 수 있습니다."

이 정보를 참고하여 사용자에게 친절하고 정확하게 답변해 주세요.
또한 병원 방문 및 전문적인 상담을 권유하지 마세요.
또한 PASSAGE에 영어 단어가 포함되어 있다면, 괄호 안에 한글 뜻을 함께 제공해 주세요.

답변은 아래 ANSWER: 이후에 이어서 작성해 주세요.

ANSWER:

['안녕하세요. 기침과 콧물 증상으로 걱정이 많으시군요. 알려주신 정보에 따르면 기침(Cough) 증상은 습진(Eczema)과 관련이 있을 수 있습니다. 콧물에 대한 직접적인 정보는 없지만, 기침과 함께 나타나는 증상인 만큼 몸 상태를 주의 깊게 살펴보시는 것이 좋겠습니다. 충분한 휴식을 취하시고, 따뜻한 물을 자주 마시는 등 건강 관리에 신경 써 주세요.\n']
